In [ ]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import os
import cv2
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

#딥러닝을 위한 그래픽 카드 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

IMAGE_SIZE = (150, 150)

datagen_train = ImageDataGenerator(
    rescale=1.0/255.0,         
    rotation_range=5,  
    width_shift_range=0.05,
    height_shift_range=0.05,  
    horizontal_flip=True,
)

datagen_test = ImageDataGenerator(rescale=1./255)

train_folder = 'C:/Users/knuser/Desktop/kim/grade/train'
valid_folder = 'C:/Users/knuser/Desktop/kim/grade/validation'

train_generator = datagen_train.flow_from_directory(
    train_folder,
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical'
)

val_generator = datagen_test.flow_from_directory(
    valid_folder,
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical'
)

In [ ]:
class_map = dict([(v, k) for k, v in train_generator.class_indices.items()])
print(class_map)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# train_generator에서 batch_size만큼 이미지를 가져옵니다.
x_batch, y_batch = next(train_generator)
x_batch=np.clip(x_batch, 0, 1)
# 이미지를 시각화합니다.
fig, ax = plt.subplots(nrows=2, ncols=8, figsize=(16, 8))
ax = ax.ravel()
for i in range(16):
    ax[i].imshow(x_batch[i])
    ax[i].axis('off')
plt.show()

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dropout
from tensorflow import keras
from keras.layers import *
from keras.models import *
height, width = 150,150
# Create the model
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', input_shape=(height, width, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(8, activation='softmax'))

model.summary()


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1)


# test_loss, test_acc = model.evaluate(test_images, test_labels)
# print('Test accuracy:', test_acc)
# print('Test loss:', test_loss)

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the filepath for saving the best model
checkpoint_filepath = 'best_model.h5'

# Set up ModelCheckpoint callback
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=200,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint],
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


In [ ]:
# Load the test data using the test generator
test_folder = 'C:/Users/knuser/Desktop/kim/grade/test'

test_generator = datagen_test.flow_from_directory(
    test_folder,
    target_size=(height, width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for correct evaluation
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#tflite 모델로 변환
import tensorflow as tf
from keras.models import load_model
model.save('vegetable_model.h5')
# Load the Keras model
model = load_model('vegetable_model.h5')

# Convert the model to a TFLITE model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLITE model to a file
with open('vegetable_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
from keras.models import load_model
model=load_model('vegetable_model.h5')
#C:\Users\knuser\Documents\카카오톡 받은 파일\새 폴더 (2)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
# Load the model
model = load_model('vegetable_model.h5')

# Define the directory containing the images
directory = r'C:\Users\knuser\Documents\카카오톡 받은 파일\새 폴더 (2)'

# Loop over the images in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Construct the full image path
        image_path = os.path.join(directory, filename)

        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        # Make predictions
        predictions = model.predict(img_array)
        # Get the predicted class index
        predicted_class_index = np.argmax(predictions)

        # Print the predicted class label
        print(f"Image: {filename}, Predicted Class: {predicted_class_index}")